In [20]:
from utils import *

In [21]:
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication(tenant_id="c643d8c8-4abb-47ba-808c-bf73f60161f1")

# Configure workspace
ws = Workspace.from_config(auth=interactive_auth)

In [22]:
from azureml.core.model import InferenceConfig

env = get_env(name="sentiment-analysis", packages=["tensorflow", "transformers"])

inference_config = InferenceConfig(
    environment=env,
    source_directory="code",
    entry_script="sentiment_analysis.py",
)

In [23]:
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=0.5, memory_gb=1, auth_enabled=True
)

In [24]:
from azureml.core import Model

service = Model.deploy(
    ws,
    "sentiment-analysis",
    [],
    inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-01-02 17:50:10+00:00 Creating Container Registry if not exists.
2022-01-02 17:50:10+00:00 Registering the environment.
2022-01-02 17:50:10+00:00 Use the existing image.
2022-01-02 17:50:10+00:00 Generating deployment configuration.
2022-01-02 17:50:11+00:00 Submitting deployment to compute.
2022-01-02 17:50:13+00:00 Checking the status of deployment sentiment-analysis..
2022-01-02 17:52:44+00:00 Checking the status of inference endpoint sentiment-analysis.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [25]:
print(service.get_logs())

2022-01-02T17:52:25,554850000+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2022-01-02T17:52:25,564100100+00:00 - rsyslog/run 
2022-01-02T17:52:25,573897700+00:00 - nginx/run 
2022-01-02T17:52:25,609152700+00:00 - iot-server/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2022-01-02T17:52:26,113825000+00:00 - iot-server/finish 1 0
2022-01-02T17:52:26,124371400+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (67)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 91
2022-01-02 17:52:28.754260: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /azureml-envs/azureml_82846cfedbce6dde324bf4713fce4fe8/lib:/azureml-envs/azureml_82846cfedbce6dde324bf4713fce4

In [26]:
import requests
import json
from azureml.core import Webservice

service = Webservice(workspace=ws, name="sentiment-analysis")
scoring_uri = service.scoring_uri

# If the service is authenticated, set the key or token
key, _ = service.get_keys()

# Set the appropriate headers
headers = {"Content-Type": "application/json"}
headers["Authorization"] = f"Bearer {key}"

data = {
    "query": "My ML skills are quite good."
}
data = json.dumps(data)
resp = requests.post(scoring_uri, data=data, headers=headers)
print(resp.text)

{"sentiment": [{"label": "POSITIVE", "score": 0.9998338222503662}]}


In [27]:
scoring_uri

'http://95b4fd8c-5b51-45db-9fbf-68bf570ca630.westus2.azurecontainer.io/score'

In [28]:
# service.delete()